In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,SimpleRNN,Dense


2025-08-29 20:25:45.977674: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-29 20:25:46.594620: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-29 20:25:49.617681: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
##Mapping of words index 
word_index = imdb.get_word_index()
reverse_word_index = {value :key for key , value in word_index.items()}

In [4]:
###Load the model
model = tf.keras.models.load_model('Simple_RNN_imdb.h5')
model.summary()

I0000 00:00:1756499270.211220   29660 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5561 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ (32, 500, 128)         │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_4 (SimpleRNN)        │ (32, 128)              │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (32, 1)                │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,027 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [7]:
model.get_weights()

[array([[-0.14155954, -0.12827931, -0.3000231 , ..., -0.00736701,
          1.0231829 ,  0.12566312],
        [-0.02274084, -0.02570872,  0.04386581, ...,  0.05644039,
          0.08160345,  0.0040139 ],
        [ 0.00155452, -0.0601109 , -0.06229394, ...,  0.03385374,
          0.22530176, -0.06634963],
        ...,
        [ 0.07265773,  0.07009377,  0.15453966, ...,  0.0882788 ,
         -0.08076608,  0.05758748],
        [ 0.00961295,  0.09079336,  0.10198012, ...,  0.00477098,
         -0.03793149,  0.07034796],
        [ 0.09801164,  0.09535129,  0.01352332, ...,  0.05875922,
         -0.01939676,  0.02929469]], shape=(10000, 128), dtype=float32),
 array([[ 0.1650656 , -0.06267059,  0.11233418, ..., -0.11762739,
         -0.09788001,  0.02669126],
        [-0.11242767,  0.13570209,  0.02892954, ..., -0.10074986,
         -0.1309563 ,  0.08642565],
        [-0.03066739,  0.12222748,  0.10047594, ..., -0.12411908,
          0.10265721, -0.08883046],
        ...,
        [-0.0775446

In [8]:
###Step 2-helper Function
def decode_review(endcoded_review):
    return ' '.join([reverse_word_index.get(i-3, '?') for i in endcoded_review])

def preprocess_review(text):
    words = text.lower().split()
    endoded_review =[word_index.get(word,2)+3 for word in words]
    padded_review = sequence.pad_sequences([endoded_review], maxlen=500)
    return padded_review


In [ ]:
###Predicton function

def predict_sentiment(review):
    preprocess = preprocess_review(review)
    models_prediction = model.predict(preprocess)
    
    sentiment = 'Positive' if models_prediction[0][0]>0.5 else 'Negative'
    return sentiment, float(models_prediction[0][0])

In [10]:
example_review = "This movie was just amazing, with stunning visuals and a gripping storyline that kept me on the edge of my seat from start to finish."

print(predict_sentiment(example_review))

2025-08-29 20:47:08.353377: I external/local_xla/xla/service/service.cc:163] XLA service 0x7508100e36a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-08-29 20:47:08.353596: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-08-29 20:47:08.425906: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-08-29 20:47:08.549094: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91200


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 991ms/step
('Positive', 0.7912697196006775)


I0000 00:00:1756500429.045837   30072 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
